In [9]:
import mlflow
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
import pandas as pd

# --- Set the MLflow Tracking URI ---
# This tells your script to communicate with the MLflow server
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# --- 1. Data Preparation ---
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=0, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_df = pd.DataFrame(X_train)

# Define an input example for the model signature
input_example = X_train_df.head(5)

# --- 2. Training and Logging ---
with mlflow.start_run() as run:
    params = {
        'objective': 'binary:logistic',
        'max_depth': 4,
        'alpha': 0.1,
        'learning_rate': 0.1,
        'n_estimators': 100
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print("Logging parameters: ", params)
    mlflow.log_params(params)

    print("Logging metric (accuracy): ", acc)
    mlflow.log_metric("accuracy", acc)

    print("\nLogging the model...")
    # Add the input_example to log the model with a signature
    mlflow.xgboost.log_model(
        xgb_model=model,
        artifact_path="model",
        input_example=input_example
    )

    run_id = run.info.run_id
    print("\n--- Model training complete ---")
    print(f"✅ Run ID: {run_id}")
    print("Copy this Run ID for the next step.")


2025/11/15 19:02:25 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Logging parameters:  {'objective': 'binary:logistic', 'max_depth': 4, 'alpha': 0.1, 'learning_rate': 0.1, 'n_estimators': 100}
Logging metric (accuracy):  0.92

Logging the model...


C:\Users\ADISH\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(



--- Model training complete ---
✅ Run ID: 904699855255461ba22d2deb0483d775
Copy this Run ID for the next step.
🏃 View run lyrical-cat-95 at: http://127.0.0.1:5000/#/experiments/0/runs/904699855255461ba22d2deb0483d775
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0


## Part 2: Register the Model

In [ ]:
import mlflow

# Tell the script where the MLflow server is
mlflow.set_tracking_uri("http://127.0.0.1:5000")

# --- 1. Provide the Run ID ---
run_id = "904699855255461ba22d2deb0483d775" # I've pre-filled this for you

# --- 2. Register the Model ---
model_name = "XGB-Smote"
model_uri = f"runs:/{run_id}/model"

print(f"\nRegistering model from URI: {model_uri}")
registered_model = mlflow.register_model(model_uri=model_uri, name=model_name)

print("\n--- Model registration complete ---")
print(f"✅ Successfully registered model '{registered_model.name}'")
print(f"   Version: {registered_model.version}")
